In [16]:
# This is a sample Python script.

# Press Shift+F10 to execute it or replace it with your code.
# Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xlrd
import csv
import os

import random
from matplotlib.collections import LineCollection
from matplotlib.lines import Line2D
import shapely.geometry as geometry
from scipy.spatial import Delaunay
from scipy.spatial import Voronoi, voronoi_plot_2d
from scipy.spatial import distance
import math
import matplotlib.tri as tri
from collections import Counter
#
#import geopandas as gpd
#from geovoronoi import voronoi_regions_from_coords
#from foronoi import Voronoi, Polygon, Visualizer, VoronoiObserver
import shapely
import math
import matplotlib.tri as tri
from collections import defaultdict
import time
import multiprocessing as mp
import torch

In [17]:
torch.cuda.is_available()

True

In [18]:
def compute_feature(folder):
    inner_path = os.path.join(outer_path, folder)
    total_num_folder = len(folderlist)
    print ('total have %d folders' % (total_num_folder))
    filelist = os.listdir(inner_path)
    path_result0 = r'/mnt/c/Users/huhui/projects/tiatoolbox/examples/TCGA-test'
    #path0 = r'/mnt/c/Users/huhui/projects/tiatoolbox/examples'
    #path_result0 = os.path.join(path0, "dep_result")
    #path_result = os.path.join(path_result0, str(folder))
    #print(path_result, end='\n')
    #if not os.path.exists(path_result):
       # mkdir(path_result)
    all_vertex = []
    all_position = []
    all_distance = []
    all_class = []
    color = []  
    #vor变量
    #all_vertexvor = []
    all_positionvor = []
    vor_class = []
    vor_area = []
    for it in range(len(filelist)):
        #print(folder)
        patch_number = len(filelist)
        print(patch_number)
        csvname = filelist[it]
        src1 = os.path.join(os.path.abspath(inner_path), filelist[it])
        print(it)
        table = xlrd.open_workbook(src1)
        #读取Tri数据并计算特征
        sheet1 = table.sheet_by_index(0)
        nrow = sheet1.nrows
        #print(nrow)
        ncols = sheet1.ncols
        title = sheet1.row_values(1)
        all_vertex0 = sheet1.col_values(1)
        list0 = [10000*it]*(nrow-1)
        #print(len(list0))
        #
            #读取vor数据并计算特征
        sheet2 = table.sheet_by_index(1)
        nrowvor = sheet2.nrows
        #ncols = sheet1.ncols
        title2 = sheet2.row_values(1)
        all_vertex1 = sheet2.col_values(1)
        all_positionb = all_vertex1[1:]
        all_positionb = [eval(x) for x in all_positionb]
        all_positionvor = all_positionb + all_positionvor 
        area0 = sheet2.col_values(2)
        vor_areaa = area0[1:]
        vor_area = vor_area + vor_areaa
        class0 = sheet2.col_values(3)
        vor_classa = class0[1:]
        vor_class = vor_classa +vor_class
        dict_vorclass = defaultdict(list)
        
        
        all_vertexa = all_vertex0[1:]
        #print(len(all_vertex))
        all_vertexa = np.sum([all_vertexa, list0], axis=0).tolist()
        all_vertex = all_vertex + all_vertexa
        #print(all_vertex)
        #all_vertex = all_vertex0[1:]+list0
        #print(all_vertex)
        position0 = sheet1.col_values(2)
        all_positiona = position0[1:]
        all_positiona = [eval(x) for x in all_positiona]
        all_position = all_position + all_positiona 
        #s = eval(position[1])
        #s = list(map(int, s))
        all_distance0 = sheet1.col_values(3)
        all_distancea = all_distance0[1:]
        all_distance = all_distance + all_distancea
        all_class0 = sheet1.col_values(4)
        all_classa = all_class0[1:]
        all_class = all_class + all_classa 
        #a = len(set(all_class))
        color0 = sheet1.col_values(5)
        colora = color0[1:]
        color = color + colora
        
    tri_number = len(all_distance)
    print(tri_number)
    tri_distance = sum(all_distance)
    tri_meandistance = np.mean(all_distance)
    tri_mediandistance = np.median(all_distance)
    tri_vardistance = np.var(all_distance)
    tri_stddistance = np.std(all_distance)
    dict_distance = defaultdict(list)
    dict_class = defaultdict(list)
    dict_color = defaultdict(list)
    dict_vectorcolor = defaultdict(list)
    for item, distancea in zip(color, all_distance):
        dict_color[item].append(distancea)
    for vertex, item in zip(all_vertex, color):
        dict_vectorcolor[vertex].append(item)

    for vertex, distancea in zip(all_vertex, all_distance):
        dict_distance[vertex].append(distancea)
    #for vertex, distance in zip(all_vertex, all_distance):
    #    dict_distance[vertex].append(distance)
    for vertex, classnum in zip(all_vertex, all_class):
        dict_class[vertex].append(classnum)
    for item in list(dict_vectorcolor.keys()):
        edge_type = dict_color.get(item)
    ver_distance = []
    ver_mix = []
    list2 = []
    for item in list(dict_distance.keys()):
        v_Dis = dict_distance.get(item)
        v_numbrt = len(v_Dis)
        vrmain_distance = np.mean(v_Dis)
        ver_distance.append(vrmain_distance)
        v_color = dict_vectorcolor.get(item)
        v_all = len(set(v_color))
        n1 = v_color.count(0)
        n2 = v_color.count(50)
        n3 = v_color.count(100)
        v_diff = v_numbrt - n1 - n2 - n3 + 1
        v_mix = v_all/(v_numbrt*v_numbrt)*v_diff
        ver_mix.append(v_mix)
        mix_mean = np.mean(ver_mix)
    mix_median = np.median(ver_mix)
    mix_var = np.var(ver_mix)
    mix_std = np.std(ver_mix)
    ditance_mean = np.mean(ver_distance)
    ditance_median = np.median(ver_distance)
    ditance_var = np.var(ver_distance)
    ditance_std = np.std(ver_distance)
    list2.append(str(folder))
    list2.append(mix_mean)
    list2.append(mix_median)
    list2.append(mix_var)
    list2.append(mix_std)
    list2.append(ditance_mean)
    list2.append(ditance_median)
    list2.append(ditance_var)
    list2.append(ditance_std)
    dict_mix0 = defaultdict(list)
    dict_mix = defaultdict(list)
    dict_compass = defaultdict(list)
    for class0, mix in zip(all_class, ver_mix):
        dict_mix[class0].append(mix)
    for class0, distance in zip(all_class, ver_distance):
        dict_compass[class0].append(distance)
    keys = sorted(dict_mix.keys(), reverse=True)
    #output_list = [dict_mix[key] for key in keys]
    #for class0, distance in zip(keys, output_list):
        #dict_mix0[class0].append(distance)
   # for i, item in enumerate(keys):
    if 3 not in keys:
            list2.append(0)
            list2.append(0)
            list2.append(0)
            list2.append(0)
            list2.append(0)
            list2.append(0)
            list2.append(0)
            list2.append(0)
    if 3 in keys:
            mixnumber = dict_mix.get(3)
            mean_mix = np.mean(mixnumber)
            distance = dict_compass.get(3)
            list2.append(np.mean(mixnumber))
            list2.append(np.median(mixnumber))
            list2.append(np.var(mixnumber))
            list2.append(np.std(mixnumber))
            list2.append(np.mean(distance))
            list2.append(np.median(distance))
            list2.append(np.var(distance))
            list2.append(np.std(distance))
    if 2 not in keys:
            list2.append(0)
            list2.append(0)
            list2.append(0)
            list2.append(0)
            list2.append(0)
            list2.append(0)
            list2.append(0)
            list2.append(0)
    if 2 in keys:
            mixnumber = dict_mix.get(2)
            mean_mix = np.mean(mixnumber)
            distance = dict_compass.get(2)
            list2.append(np.mean(mixnumber))
            list2.append(np.median(mixnumber))
            list2.append(np.var(mixnumber))
            list2.append(np.std(mixnumber))
            list2.append(np.mean(distance))
            list2.append(np.median(distance))
            list2.append(np.var(distance))
            list2.append(np.std(distance))
    if 1 not in keys:
            list2.append(0)
            list2.append(0)
            list2.append(0)
            list2.append(0)
            list2.append(0)
            list2.append(0)
            list2.append(0)
            list2.append(0)
    if 1 in keys:
            mixnumber = dict_mix.get(1)
            mean_mix = np.mean(mixnumber)
            distance = dict_compass.get(1)
            list2.append(np.mean(mixnumber))
            list2.append(np.median(mixnumber))
            list2.append(np.var(mixnumber))
            list2.append(np.std(mixnumber))
            list2.append(np.mean(distance))
            list2.append(np.median(distance))
            list2.append(np.var(distance))
            list2.append(np.std(distance))
    list1 = ['mix_mean', 'mix_median', 'mix_var', 'mix_std', 'compass_mean', 'compass_median', 'compass_var', 'compass_std', 'ttmix_mean', 'ttmix_median', 'ttmix_var', 'ttmix_std', 'ttcompass_mean', 'ttcompass_median', 'ttcompass_var', 'ttcompass_std',
             'ssmix_mean', 'ssmix_median', 'ssmix_var', 'ssmix_std', 'sscompass_mean', 'sscompass_median', 'sscompass_var', 'sscompass_std',
             'llmix_mean', 'llmix_median', 'llmix_var', 'llmix_std', 'llcompass_mean', 'llcompass_median', 'llcompass_var', 'llcompass_std',
             'tt_number', 'tt_frequency', 'tt_distance', 'tt_meandistance', 'tt_mediandistance', 'tt_vardistance',
             'tt_stddistance', 'll_number', 'll_frequency', 'll_distance', 'll_meandistance', 'll_mediandistance',
             'll_vardistance', 'll_stddistance',
             'ss_number', 'ss_frequency', 'ss_distance', 'ss_meandistance', 'ss_mediandistance', 'ss_vardistance',
             'ss_stddistance', 'ts_number', 'ts_frequency', 'ts_distance', 'ts_meandistance', 'ts_mediandistance',
             'ts_vardistance', 'ts_stddistance',
             'sl_number', 'sl_frequency', 'sl_distance', 'sl_meandistance', 'sl_mediandistance', 'sl_vardistance',
             'sl_stddistance', 'tl_number', 'tl_frequency', 'tl_distance', 'tl_meandistance', 'tl_mediandistance',
             'tl_vardistance', 'tl_stddistance',
             'tri_number', 'tri_distance', 'tri_meandistance', 'tri_mediandistance', 'tri_vardistance', 'tri_stddistance']


    tumor_tumor = dict_color.get(0)
    tt_number = len(tumor_tumor)
    tt_distance = sum(tumor_tumor)
    tt_meandistance = tt_distance/tt_number
    #0:t-t 150: t-s 100:s-s  255:t-l  50:l-l 200:s-l
    #list1 = ['tt_number', 'tt_frequency', 'tt_distance', 'tt_meandistance', 'tt_mediandistance', 'tt_vardistance', 'tt_stddistance', 'ts_number', 'ts_frequency', 'ts_distance', 'ts_meandistance', 'ts_mediandistance', 'ts_vardistance', 'ts_stddistance',
             #'ss_number', 'ss_frequency', 'ss_distance', 'ss_meandistance', 'ss_mediandistance', 'ss_vardistance', 'ss_stddistance', 'tl_number', 'tl_frequency', 'tl_distance', 'tl_meandistance', 'tl_mediandistance', 'tl_vardistance', 'tl_stddistance',
             #'ll_number', 'll_frequency', 'll_distance', 'll_meandistance', 'll_mediandistance', 'll_vardistance', 'll_stddistance', 'sl_number', 'sl_frequency', 'sl_distance', 'sl_meandistance', 'sl_mediandistance', 'sl_vardistance', 'sl_stddistance',
             #'tri_number', 'tri_distance', 'tri_meandistance', 'tri_mediandistance', 'tri_vardistance', 'tri_stddistance']
    #dic1 = dict.fromkeys(list1)'tt_meandistance',
    #list7 = ['tt_number', 'tt_frequency', 'tt_distance', 'tt_meandistance', 'tt_mediandistance', 'tt_vardistance', 'tt_stddistance', 'll_number', 'll_frequency', 'll_distance', 'll_meandistance', 'll_mediandistance', 'll_vardistance', 'll_stddistance',
             #'ss_number', 'ss_frequency', 'ss_distance', 'ss_meandistance', 'ss_mediandistance', 'ss_vardistance', 'ss_stddistance', 'ts_number', 'ts_frequency', 'ts_distance', 'ts_meandistance', 'ts_mediandistance', 'ts_vardistance', 'ts_stddistance',
             #'sl_number', 'sl_frequency', 'sl_distance', 'sl_meandistance', 'sl_mediandistance', 'sl_vardistance', 'sl_stddistance', 'tl_number', 'tl_frequency', 'tl_distance', 'tl_meandistance', 'tl_mediandistance', 'tl_vardistance', 'tl_stddistance']
    colorall = [0, 50, 100, 150, 200, 255]
    keys_color = sorted(dict_color.keys())
    for item in colorall:
        if item in keys_color:
            edge_type = dict_color.get(item)
            number = len(edge_type)
            list2.append(number)
            edge_frequency = number/tri_number
            list2.append(edge_frequency)
            distance = sum(edge_type)
            list2.append(distance)
            mean_distance = np.mean(edge_type)
            list2.append(mean_distance)
            median_distance = np.median(edge_type)
            list2.append(median_distance)
            var_distance = np.var(edge_type)
            list2.append(var_distance)
            std_distance = np.std(edge_type)
            list2.append(std_distance)
        if item not in keys_color:
            list2.append(0)
            list2.append(0)
            list2.append(0)
            list2.append(0)
            list2.append(0)
            list2.append(0)
            list2.append(0)
    list2.append(tri_number)
    list2.append(tri_distance)
    list2.append(tri_meandistance)
    list2.append(tri_mediandistance)
    list2.append(tri_vardistance)
    list2.append(tri_stddistance)
    dict1 = dict(zip(list1,list2))
    
    #读取vor数据并计算特征
    ##sheet2 = table.sheet_by_index(1)
    #nrow = sheet2.nrows
    #ncols = sheet2.ncols
   ## title2 = sheet2.row_values(1)
    ##all_vertex0 = sheet2.col_values(1)
   ## all_positon = all_vertex0[1:]
   ## all_positon = [eval(x) for x in all_positon]
   ## area0 = sheet2.col_values(2)
   ## vor_area = area0[1:]
   ## class0 = sheet2.col_values(3)
   ## vor_class = class0[1:]
   ## dict_vorclass = defaultdict(list)
    
    
    
    vor_number = len(vor_area)
    mean_vor = np.mean(vor_area)
    var_vor = np.var(vor_area)
    std_vor = np.std(vor_area)
    compass_vor = std_vor/mean_vor
    for item, area in zip(vor_class, vor_area):
        dict_vorclass[item].append(area)
    list4 = []
    list3 = ['tumor_number', 'tumor_density', 'tumor_area', 'tumor_mean', 'tumor_median', 'tumor_var', 'tumor_std', 'tumor_compass',
             'stormal_number', 'stormal_density', 'stormal_area', 'stormal_mean', 'stormal_median', 'stormal_var', 'stormal_std', 'stormal_compass',
             'lym_number', 'lym_density', 'lym_area',  'lym_mean', 'lym_median', 'lym_var', 'lym_std', 'lym_compass',
             'vor_number', 'mean_vor', 'var_vor', 'std_vor', 'compass_vor']
    #3肿瘤2基质1淋巴
    classall = [3, 2, 1]
    keys_vorclass = sorted(dict_vorclass.keys(), reverse=True)
    for item in classall:
        if item in keys_vorclass:
            vor_type = dict_vorclass.get(item)
            number = len(vor_type)
            list4.append(number)
            density_vor = number / vor_number
            list4.append(density_vor)
            area = sum(vor_type)
            list4.append(area)
            mean_area = np.mean(vor_type)
            list4.append(mean_area)
            median_area = np.median(vor_type)
            list4.append(median_area)
            var_area = np.var(vor_type)
            list4.append(var_area)
            std_area = np.std(vor_type)
            list4.append(std_area)
            compass = std_area / mean_area
            list4.append(compass)
        if item not in keys_vorclass:
            list4.append(0)
            list4.append(0)
            list4.append(0)
            list4.append(0)
            list4.append(0)
            list4.append(0)
            list4.append(0)
            list4.append(0)
    
    
    list4.append(vor_number)
    list4.append(mean_vor)
    list4.append(var_vor)
    list4.append(std_vor)
    list4.append(compass_vor)
    dict2 = dict(zip(list3, list4))
    dictaa = dict(dict1, **dict2)
    listnew = list2+list4
    #saveDict = {"用户1": "密码1"，"用户2": "密码2"}
    return listnew

In [19]:
if __name__ == '__main__':
    #table = xlrd.open_workbook(r'432831A4_7_Info.xlsx')
    start = time.time()
    outer_path = '/mnt/c/Users/huhui/projects/tiatoolbox/examples/TCGAHCC-result'
    # Root directory of the project
    ROOT_DIR = os.getcwd()
    print(ROOT_DIR)
    it=1
    folderlist = os.listdir(outer_path)
    fileName = "TCGA-test.csv"
    name_attribute = ['patientID', 'mix_mean', 'mix_median', 'mix_var', 'mix_std', 'compass_mean', 'compass_median', 'compass_var', 'compass_std', 'ttmix_mean', 'ttmix_median', 'ttmix_var', 'ttmix_std', 'ttcompass_mean', 'ttcompass_median', 'ttcompass_var', 'ttcompass_std',
            'ssmix_mean', 'ssmix_median', 'ssmix_var', 'ssmix_std', 'sscompass_mean', 'sscompass_median', 'sscompass_var', 'sscompass_std',
            'llmix_mean', 'llmix_median', 'llmix_var', 'llmix_std', 'llcompass_mean', 'llcompass_median', 'llcompass_var', 'llcompass_std','tt_number', 'tt_frequency', 'tt_distance', 'tt_meandistance', 'tt_mediandistance',
                          'tt_vardistance',
                          'tt_stddistance', 'ts_number', 'ts_frequency', 'ts_distance', 'ts_meandistance',
                          'ts_mediandistance',
                          'ts_vardistance', 'ts_stddistance',
                          'ss_number', 'ss_frequency', 'ss_distance', 'ss_meandistance', 'ss_mediandistance',
                          'ss_vardistance',
                          'ss_stddistance', 'tl_number', 'tl_frequency', 'tl_distance', 'tl_meandistance',
                          'tl_mediandistance',
                          'tl_vardistance', 'tl_stddistance',
                          'll_number', 'll_frequency', 'll_distance', 'll_meandistance', 'll_mediandistance',
                          'll_vardistance',
                          'll_stddistance', 'sl_number', 'sl_frequency', 'sl_distance', 'sl_meandistance',
                          'sl_mediandistance',
                          'sl_vardistance', 'sl_stddistance',
                          'tri_number', 'tri_distance', 'tri_meandistance', 'tri_mediandistance', 'tri_vardistance',
                          'tri_stddistance', 'tumor_number', 'tumor_density', 'tumor_area',  'tumor_mean',
                          'tumor_median', 'tumor_var', 'tumor_std', 'tumor_compass',
                          'stormal_number', 'stormal_density', 'stormal_area', 'stormal_mean',
                          'stormal_median', 'stormal_var', 'stormal_std', 'stormal_compass',
                          'lym_number', 'lym_density', 'lym_area', 'lym_mean', 'lym_median', 'lym_var',
                          'lym_std', 'lym_compass',
                          'vor_number', 'mean_vor', 'var_vor', 'std_vor', 'compass_vor']
    #tt = ["a", "b"]
    
    
    with open(fileName, "w", newline='') as csv_file:
           writer = csv.writer(csv_file)
           #for key, value in dictaa.items:
           writer.writerow(name_attribute)
    
           for folder in folderlist:
                results = compute_feature(folder)  
                writer.writerow(results)
        
        ##保存文件
        #s=dictaa.items
        #with open(fileName, "w") as csv_file:
            #writer = csv.writer(csv_file)
            #for key, value in dictaa.items:
            #writer.writerow(results)
        #fileObject = open('/mnt/c/Users/huhui/projects/tiatoolbox/examples/feature_result/TCGA-2Y-A9GV-01Z-00-DX1.524FA6DD-7C84-425C-A2B5-17690DF50A28_1_Info.csv', 'a')

        # dataini = pd.DataFrame(columns=('numberx', 'numbery'))
        #writerCSV = pd.DataFrame(columns=name_attribute)
        # data = {'NumberID': list_inner, 'voro_area': voro_area, 'type_area': type_area}
        #dataini = writerCSV._append(pd.DataFrame(results, index=[0]))
        #dataini.to_csv(fileObject)
        #fileObject.close()


    print(time.time()-start)

/mnt/c/Users/huhui/projects/tiatoolbox/examples
total have 1 folders
4
0
4
1
4
2
4
3
191244
43.88344955444336
